In [1]:
import kaggle as kg
import os
import pathlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
from keras.utils import to_categorical

from keras.layers import Flatten, Dense
from keras.applications import VGG16
from keras.models import Model
from keras.optimizers import SGD

2024-07-23 00:00:05.391384: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-07-23 00:00:05.943271: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-07-23 00:00:05.946802: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-07-23 00:00:08.551245: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [ ]:
os.environ["KAGGLE_USERNAME"] = "aimlrl"
os.environ["KAGGLE_KEY"] = "54d4150a6ca782d7b27af3f3754eddd8"

In [ ]:
kg.api.authenticate()

In [ ]:
kg.api.dataset_download_files(dataset="ayuraj/asl-dataset",
                              path="dataset",unzip=True)

In [2]:
def train_test_df(path):

    img_path = list()
    img_label = list()

    for single_class_dir_path in pathlib.Path(path).glob("*"):

        if str(single_class_dir_path).split("/")[-1] == "asl_dataset":
            continue

        for single_class_img_path in pathlib.Path(single_class_dir_path).glob("*.jpeg"):

            img_path.append(str(single_class_img_path))
            img_label.append(str(single_class_img_path).split("/")[-2])

    return pd.DataFrame(data={"img_path":img_path,"label":img_label})        

In [3]:
data_path = "dataset/asl_dataset"

In [4]:
data = train_test_df(data_path)

In [5]:
data

,img_path,label
0,dataset/asl_dataset/g/hand1_g_top_seg_3_croppe...,g
1,dataset/asl_dataset/g/hand1_g_left_seg_4_cropp...,g
2,dataset/asl_dataset/g/hand1_g_bot_seg_2_croppe...,g
3,dataset/asl_dataset/g/hand5_g_dif_seg_4_croppe...,g
4,dataset/asl_dataset/g/hand1_g_bot_seg_3_croppe...,g
...,...,...
2510,dataset/asl_dataset/a/hand1_a_bot_seg_2_croppe...,a
2511,dataset/asl_dataset/a/hand2_a_left_seg_4_cropp...,a
2512,dataset/asl_dataset/a/hand2_a_right_seg_4_crop...,a
2513,dataset/asl_dataset/a/hand2_a_top_seg_2_croppe...,a


In [6]:
for label in data['label'].unique():

    print(data[data['label'] == label].shape[0])

70
70
70
70
70
70
70
70
70
70
70
70
70
70
70
70
70
70
70
70
70
70
70
70
70
70
70
70
70
70
70
70
70
65
70
70


In [7]:
training_data = pd.DataFrame(data={"img_path":[],"label":[]})
testing_data = pd.DataFrame(data={"img_path":[],"label":[]})

In [8]:
for label in data["label"].unique():

    training_data = pd.concat((training_data,data[data["label"] == label].iloc[0:60]),axis=0)
    testing_data = pd.concat((testing_data,data[data["label"] == label].iloc[60:]),axis=0)

In [9]:
training_data

,img_path,label
0,dataset/asl_dataset/g/hand1_g_top_seg_3_croppe...,g
1,dataset/asl_dataset/g/hand1_g_left_seg_4_cropp...,g
2,dataset/asl_dataset/g/hand1_g_bot_seg_2_croppe...,g
3,dataset/asl_dataset/g/hand5_g_dif_seg_4_croppe...,g
4,dataset/asl_dataset/g/hand1_g_bot_seg_3_croppe...,g
...,...,...
2500,dataset/asl_dataset/a/hand2_a_right_seg_1_crop...,a
2501,dataset/asl_dataset/a/hand1_a_top_seg_2_croppe...,a
2502,dataset/asl_dataset/a/hand1_a_bot_seg_5_croppe...,a
2503,dataset/asl_dataset/a/hand1_a_left_seg_5_cropp...,a


In [10]:
testing_data

,img_path,label
60,dataset/asl_dataset/g/hand1_g_bot_seg_5_croppe...,g
61,dataset/asl_dataset/g/hand2_g_dif_seg_3_croppe...,g
62,dataset/asl_dataset/g/hand5_g_bot_seg_1_croppe...,g
63,dataset/asl_dataset/g/hand3_g_dif_seg_1_croppe...,g
64,dataset/asl_dataset/g/hand1_g_right_seg_3_crop...,g
...,...,...
2510,dataset/asl_dataset/a/hand1_a_bot_seg_2_croppe...,a
2511,dataset/asl_dataset/a/hand2_a_left_seg_4_cropp...,a
2512,dataset/asl_dataset/a/hand2_a_right_seg_4_crop...,a
2513,dataset/asl_dataset/a/hand2_a_top_seg_2_croppe...,a


In [11]:
training_data.reset_index(inplace=True,drop=True)

In [12]:
training_data

,img_path,label
0,dataset/asl_dataset/g/hand1_g_top_seg_3_croppe...,g
1,dataset/asl_dataset/g/hand1_g_left_seg_4_cropp...,g
2,dataset/asl_dataset/g/hand1_g_bot_seg_2_croppe...,g
3,dataset/asl_dataset/g/hand5_g_dif_seg_4_croppe...,g
4,dataset/asl_dataset/g/hand1_g_bot_seg_3_croppe...,g
...,...,...
2155,dataset/asl_dataset/a/hand2_a_right_seg_1_crop...,a
2156,dataset/asl_dataset/a/hand1_a_top_seg_2_croppe...,a
2157,dataset/asl_dataset/a/hand1_a_bot_seg_5_croppe...,a
2158,dataset/asl_dataset/a/hand1_a_left_seg_5_cropp...,a


In [13]:
testing_data.reset_index(drop=True,inplace=True)

In [14]:
testing_data

,img_path,label
0,dataset/asl_dataset/g/hand1_g_bot_seg_5_croppe...,g
1,dataset/asl_dataset/g/hand2_g_dif_seg_3_croppe...,g
2,dataset/asl_dataset/g/hand5_g_bot_seg_1_croppe...,g
3,dataset/asl_dataset/g/hand3_g_dif_seg_1_croppe...,g
4,dataset/asl_dataset/g/hand1_g_right_seg_3_crop...,g
...,...,...
350,dataset/asl_dataset/a/hand1_a_bot_seg_2_croppe...,a
351,dataset/asl_dataset/a/hand2_a_left_seg_4_cropp...,a
352,dataset/asl_dataset/a/hand2_a_right_seg_4_crop...,a
353,dataset/asl_dataset/a/hand2_a_top_seg_2_croppe...,a


In [15]:
character2int = dict(zip(training_data["label"].unique(),range(len(training_data["label"].unique()))))

In [16]:
training_data["label"].replace(to_replace=character2int.keys(),value=character2int.values(),
                               inplace=True)

testing_data["label"].replace(to_replace=character2int.keys(),value=character2int.values(),
                     inplace=True)

/tmp/ipykernel_57839/3346507411.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  training_data["label"].replace(to_replace=character2int.keys(),value=character2int.values(),
/tmp/ipykernel_57839/3346507411.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  training_data["label"].replace(to_repla

In [17]:
training_data

,img_path,label
0,dataset/asl_dataset/g/hand1_g_top_seg_3_croppe...,0
1,dataset/asl_dataset/g/hand1_g_left_seg_4_cropp...,0
2,dataset/asl_dataset/g/hand1_g_bot_seg_2_croppe...,0
3,dataset/asl_dataset/g/hand5_g_dif_seg_4_croppe...,0
4,dataset/asl_dataset/g/hand1_g_bot_seg_3_croppe...,0
...,...,...
2155,dataset/asl_dataset/a/hand2_a_right_seg_1_crop...,35
2156,dataset/asl_dataset/a/hand1_a_top_seg_2_croppe...,35
2157,dataset/asl_dataset/a/hand1_a_bot_seg_5_croppe...,35
2158,dataset/asl_dataset/a/hand1_a_left_seg_5_cropp...,35


In [18]:
testing_data

,img_path,label
0,dataset/asl_dataset/g/hand1_g_bot_seg_5_croppe...,0
1,dataset/asl_dataset/g/hand2_g_dif_seg_3_croppe...,0
2,dataset/asl_dataset/g/hand5_g_bot_seg_1_croppe...,0
3,dataset/asl_dataset/g/hand3_g_dif_seg_1_croppe...,0
4,dataset/asl_dataset/g/hand1_g_right_seg_3_crop...,0
...,...,...
350,dataset/asl_dataset/a/hand1_a_bot_seg_2_croppe...,35
351,dataset/asl_dataset/a/hand2_a_left_seg_4_cropp...,35
352,dataset/asl_dataset/a/hand2_a_right_seg_4_crop...,35
353,dataset/asl_dataset/a/hand2_a_top_seg_2_croppe...,35


In [19]:
plt.imread(training_data.iloc[0,0]).shape

(400, 400, 3)

In [20]:
plt.imread(testing_data.iloc[0,0]).shape

(400, 400, 3)

In [21]:
Y_true_train = to_categorical(y=training_data["label"],num_classes=36)
Y_true_test = to_categorical(y=testing_data["label"],num_classes=36)

In [22]:
def multiclass_cnn():

    vgg16 = VGG16(include_top=False,input_shape=(400,400,3),weights=None,pooling="avg")
    input_to_vgg16 = vgg16.layers[0].input
    vgg16_output = Dense(units=36,activation="softmax")(vgg16.layers[-1].output)

    return Model(inputs=[input_to_vgg16],outputs=[vgg16_output])

In [23]:
def custom_data_generator(data_df, Y_true, mb_size):

    for time_step in range(data_df.shape[0]//mb_size):
        X_mb = list()

        for img_path in data_df.iloc[time_step*mb_size:(time_step+1)*mb_size,0]:

            img_np_array = plt.imread(img_path)
            #reshaped_np_array = img_np_array.reshape(1024,)
            X_mb.append(img_np_array)

        X_mb = np.array(X_mb)
        Y_true_mb = Y_true[time_step*mb_size:(time_step+1)*mb_size]

        yield X_mb, Y_true_mb

In [24]:
epochs = 50
training_data_mb_size = 3
testing_data_mb_size = 5

In [25]:
model = multiclass_cnn()
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 400, 400, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 400, 400, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 400, 400, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 200, 200, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 200, 200, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 200, 200, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 100, 100, 128)     0     

In [26]:
def loss_fn(Y_true_mb,Y_pred_mb):

    return tf.reduce_mean(tf.keras.losses.categorical_crossentropy(y_true=Y_true_mb,
                                                                          y_pred=Y_pred_mb))

optimizer = SGD()

In [27]:
@tf.function
def training_step(X_train_mb,Y_true_train_mb):

    with tf.GradientTape() as tape:
            
        Y_pred_train_mb = model(X_train_mb, training=True)
        training_loss = loss_fn(Y_true_train_mb, Y_pred_train_mb)

    grads = tape.gradient(training_loss, model.trainable_weights)
    optimizer.apply_gradients(zip(grads, model.trainable_weights))

    train_acc_metric.update_state(Y_true_train_mb,Y_pred_train_mb)

    return training_loss

In [28]:
@tf.function
def testing_forward_pass(X_test_mb,Y_true_test_mb):

    Y_pred_test_mb = model(X_test_mb,training=False)
    testing_loss = loss_fn(Y_true_test_mb,Y_pred_test_mb)
    test_acc_metric.update_state(Y_true_test_mb,Y_pred_test_mb)

    return testing_loss

In [29]:
train_acc_metric = tf.keras.metrics.CategoricalAccuracy()
test_acc_metric = tf.keras.metrics.CategoricalAccuracy()

for epoch in range(epochs):

    training_data_generator = custom_data_generator(training_data,Y_true_train,training_data_mb_size)

    for time_step, (X_train_mb, Y_true_train_mb) in enumerate(training_data_generator):
        training_loss = training_step(X_train_mb,Y_true_train_mb)

        if (time_step+1) % 50 == 0:
            print("Epoch %d, Time Step %d, Training loss for one mini batch: %.4f"
            % (epoch+1, time_step+1, float(training_loss)))
            
    training_acc = train_acc_metric.result()    
    print("Epoch %d, Training Accuracy: %.2f" % (epoch+1,float(training_acc)))
    train_acc_metric.reset_states()

    testing_data_generator = custom_data_generator(testing_data,Y_true_test,testing_data_mb_size)

    for X_test_mb, Y_true_test_mb in testing_data_generator:
        testing_loss = testing_forward_pass(X_test_mb,Y_true_test_mb)

    print("\nEpoch %d, Testing Loss for last mini batch: %.4f" % (epoch+1,float(testing_loss)))
    testing_acc = test_acc_metric.result()
    print("Epoch %d, Testing Accuracy: %.2f" % (epoch+1,float(testing_acc)))
    test_acc_metric.reset_states()

    print("\n\n")

2024-07-23 00:00:54.338734: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 122880000 exceeds 10% of free system memory.
2024-07-23 00:00:54.476861: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 122880000 exceeds 10% of free system memory.
2024-07-23 00:00:57.686410: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 92160000 exceeds 10% of free system memory.
2024-07-23 00:00:57.691227: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 92160000 exceeds 10% of free system memory.
2024-07-23 00:00:57.901211: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 92160000 exceeds 10% of free system memory.


: 